In [1]:
#pip install pyngrok==4.1.1
#pip install flask-ngrok

In [2]:
# 2Cgo3ysthiZF4UGRmtAViko3bWr_4eC9s2WRv1kEWame8xih
#!ngrok authtoken 2Cgo3ysthiZF4UGRmtAViko3bWr_4eC9s2WRv1kEWame8xih

In [3]:
"""## 1. 匯入套件"""
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import pandas as pd

import os
#import cv2
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [4]:
"""## 2. 建立模型"""

resnet_model_food_menu = ResNet50(weights="imagenet", include_top=False, input_shape=(160, 160, 3))

model_food_menu = Sequential()
model_food_menu.add(resnet_model_food_menu)
model_food_menu.add(Dropout(0.5))
model_food_menu.add(GlobalAveragePooling2D())
model_food_menu.add(Dropout(0.5))
model_food_menu.add(Dense(3, activation="softmax"))
model_food_menu.summary()   # 顯示模型摘要資訊

resnet_model_food_menu.trainable = False

"""## 3. 匯入權重"""

model_food_menu.load_weights("食物菜單_160_160_best_model.h5")

model_food_menu.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 dropout (Dropout)           (None, 5, 5, 2048)        0         
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 3)                 6147      
                                                                 
Total params: 23,593,859
Trainable params: 23,540,739
Non-trainable params: 53,120
_______________________________________

In [5]:
resnet_model_food_categorical = ResNet50(weights="imagenet", include_top=False, input_shape=(200, 200, 3))

model_food_categorical = Sequential()
model_food_categorical.add(resnet_model_food_categorical)
model_food_categorical.add(Dropout(0.5))
model_food_categorical.add(GlobalAveragePooling2D())
model_food_categorical.add(Dense(50))
model_food_categorical.add(Dropout(0.5))
model_food_categorical.add(Dense(18, activation="softmax"))
model_food_categorical.summary()   # 顯示模型摘要資訊

resnet_model_food_categorical.trainable = False

model_food_categorical.load_weights("食物類別_200_200_best_model.h5")

model_food_categorical.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

num_of_label = {'bawan': 0, 'beef_noodles': 1, 'braised_pork_over_rice': 2, 'bubble_tea': 3, 'coffin_toast': 4, 'deep-fried_chicken_cutlets': 5, 'grilled_taiwanese_sausage': 6, 'hot_sour_soup': 7, 'intestine_and_oyster_vermicelli': 8, 'milkfish_belly_congee': 9, 'oyster_omelet': 10, 'pig_s_blood_soup': 11, 'pineapple_cake': 12, 'potsticker': 13, 'rice_dumpling': 14, 'stinky_tofu': 15, 'taro_ball': 16, 'xiaolongbao': 17}
eng_ch_label_dict = pd.read_csv("食物分類_中英對照_18.csv", index_col=0, squeeze=True).to_dict()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 dropout_2 (Dropout)         (None, 7, 7, 2048)        0         
                                                                 
 global_average_pooling2d_1   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_1 (Dense)             (None, 50)                102450    
                                                                 
 dropout_3 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 18)                918       
                                                      

In [9]:
def pre_food_menu(img):
    img_np = np.empty((1,160,160,3))
    img_np[0] = img
    img_np = preprocess_input(img_np)
    result = model_food_menu.predict(img_np)
    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print("全部預測結果：")
    print(result, end = "\n")
    maxindex = np.argmax(result, axis = 1)
    print("個別結果：")
    for index, i in enumerate(maxindex):
        print("第" + str(index + 1) + "張：", end = "")
        if i == 0:
            print("食物")
            return ("食物")
        if i == 1:
            print("非食物")
            return ("非食物")
        if i == 2:
            print("菜單")
            return ("菜單")

In [10]:
def pre_food_categorical(img):
    img_np = np.empty((1,200,200,3))
    img_np[0] = img
    img_np = preprocess_input(img_np)
    result = model_food_categorical.predict(img_np)
    np.set_printoptions(formatter={'float': '{: 0.5f}'.format})
    print("全部預測結果：")
    print(result, end = "\n")
    maxindex = np.argmax(result, axis = 1)
    print("個別結果：")
    for index, i in enumerate(maxindex):
        print("第" + str(index + 1) + "張：", end = "")
        key = [k for k, v in num_of_label.items() if v == maxindex][0]
        ch_key = eng_ch_label_dict[key]
        print(ch_key)
        return ch_key

In [13]:
from IPython.core.display import Image
from flask import Flask
from flask import request
#from flask_ngrok import run_with_ngrok

from PIL import Image

app = Flask(__name__)
#run_with_ngrok(app)

@app.route('/food_menu', methods = ['GET', 'POST'])
def food_menu():
    if request.method == 'GET':
        return """
        <form method='post' action='' enctype="multipart/form-data">
            <p><input type='file' required name='image' accept= "image/gif, image/jpeg, image/jpg, image/png"></p>
            <p><button type='submit'>確定</button></p>
        </form>
        """
    else:
        try:
            img = Image.open(request.files['image'].stream).resize((160, 160)).convert('RGB')
            result = pre_food_menu(img)
            
            return result
        except Exception as e:
            print("錯誤：", e)
            return e
        
@app.route('/food_categorical', methods = ['GET', 'POST'])
def food_categorical():
    if request.method == 'GET':
        return """
        <form method='post' action='' enctype="multipart/form-data">
            <p><input type='file' required name='image' accept= "image/gif, image/jpeg, image/jpg, image/png"></p>
            <p><button type='submit'>確定</button></p>
        </form>
        """
    else:
        try:
            img = Image.open(request.files['image'].stream).resize((200, 200)).convert('RGB')
            result = pre_food_categorical(img)
            
            return result
        except Exception as e:
            print("錯誤：", e)
            return e

if __name__ == '__main__':
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://192.168.0.17:5000/ (Press CTRL+C to quit)
